# ⚡ Energía y Recursos Naturales: continuidad, costos y seguridad
KPIs: factor de planta, OEE, pérdidas no técnicas, incidentes.

## Casos de uso
- Mantenimiento predictivo en equipos críticos
- Balance energético y detección de pérdidas
- Optimización de rutas/logística en upstream/downstream
- Monitoreo ambiental y cumplimiento regulatorio

In [ ]:
# Esbozo de alerta por sensor
def alerta(temp, umbral=80):
    return 'ALERTA' if temp>umbral else 'OK'
[alerta(t) for t in [65, 78, 85]]

## Arquitectura
- Ingesta IoT/SCADA streaming, almacenamiento time-series
- Feature engineering en ventana y gemelos digitales
- Integración con sistemas de seguridad y HSE

## 💼 Valor en Energía/Recursos
- Menos paradas no programadas (uptime ↑) y OEE ↑.
- Pérdidas técnicas/no técnicas ↓ con monitoreo y balance energético.
- Cumplimiento HSE y ambiental con datos trazables.


In [ ]:
# Práctica: sensor y alerta (toy)
import pandas as pd
import numpy as np
np.random.seed(0)
serie = pd.Series(70 + np.random.randn(50)*5)
movil = serie.rolling(5).mean()
alertas = (serie > 80).sum()
print('Temp promedio:', round(serie.mean(),1))
print('Ventana(5) promedio última:', round(movil.iloc[-1],1))
print('Alertas >80°C:', int(alertas))


## 🏗️ Arquitectura IoT/Time-Series
- Ingesta SCADA/IoT en streaming, almacenamiento TSDB/lakehouse.
- Feature engineering en ventana, gemelos digitales y mantenimiento predictivo.
- Integración con HSE y sistemas de seguridad.
